Kodun Genel Amacı
Bu kod, bir A/B testi (örneğin, bir e-ticaret sitesinde "Sepete Ekle" butonunun iki farklı versiyonunun performansını karşılaştırma) için veri analizi yapmak amacıyla yazılmış. İşlevleri:

Sentetik veri üretimi: Gerçekçi kullanıcı verileri oluşturur (yaş, cinsiyet, cihaz, dönüşüm, gelir vb.).
Veri kalite kontrolü: Verilerde eksiklik, tekrar veya dengesizlik olup olmadığını kontrol eder.
İstatistiksel testler: A ve B grupları arasında dönüşüm oranı, sayfa kalış süresi ve gelir farklarını analiz eder.
Makine öğrenmesi: Dönüşüm olasılığını tahmin etmek için modeller eğitir.
Görselleştirmeler: Test sonuçlarını görselleştirir (örneğin, dönüşüm oranı çubuk grafiği).
İş metrikleri: Dönüşüm artışı, kullanıcı başına gelir ve yatırım getirisi (ROI) hesaplar.
Unit testler: Kodun doğru çalıştığını doğrulamak için testler içerir.
Kod, Google Colab gibi bir ortamda çalıştırılmak üzere tasarlanmış ve sonuçları JSON dosyasına kaydeder, görselleştirmeleri HTML olarak indirir.

In [ ]:
# Gerekli kütüphaneleri yükler
!pip install pandas numpy matplotlib seaborn plotly scikit-learn statsmodels imbalanced-learn
!pip install git+https://github.com/Matt52/bayesian-testing.git

# Kütüphaneleri içe aktarma: veri işleme, görselleştirme, istatistik ve makine öğrenmesi için
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
import json
import os
from datetime import datetime, timedelta
from typing import Dict, Tuple, Optional, List
from scipy import stats
from scipy.stats import chi2_contingency, ttest_ind, mannwhitneyu
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from statsmodels.stats.power import TTestIndPower
import unittest

# Uyarı mesajlarını gizleme
warnings.filterwarnings('ignore')

# NumPy veri tiplerini JSON uyumlu Python tiplerine dönüştürme
def make_json_serializable(obj):
    # NumPy boolean'ı Python boolean'ına çevirme
    if isinstance(obj, np.bool_):
        return bool(obj)
    # NumPy tamsayısını Python tamsayısına çevirme
    if isinstance(obj, np.integer):
        return int(obj)
    # NumPy ondalık sayısını Python ondalık sayısına çevirme
    if isinstance(obj, np.floating):
        return float(obj)
    # NumPy dizisini Python listesine çevirme
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    # Sözlük içindeki değerleri dönüştürme
    if isinstance(obj, dict):
        return {k: make_json_serializable(v) for k, v in obj.items()}
    # Liste içindeki elemanları dönüştürme
    if isinstance(obj, list):
        return [make_json_serializable(item) for item in obj]
    return obj

# A/B testi analizini yapan ana sınıf
class ABTestAnalyzer:
    def __init__(self, config: Optional[Dict] = None):
        # Veri ve sonuçlar için boş değişkenler tanımları
        self.data = None
        self.results = {}
        # Kullanıcı yapılandırmasını veya varsayılan ayarları yükleme
        self.config = config or self._get_default_config()
        # Görselleştirme tarzını ayarlama
        plt.style.use('seaborn-v0_8-darkgrid')
        sns.set_palette("husl")
        print(" A/B Test Analyzer başlatıldı!")
        print(f" Konfigürasyon: {len(self.config)} ayar yüklendi")

    # Varsayılan yapılandırma ayarlarını döndürme
    def _get_default_config(self) -> Dict:
        return {
            'alpha_level': 0.05,  # İstatistiksel anlamlılık eşiği (%5)
            'power': 0.8,  # Test gücü (%80)
            'minimum_effect_size': 0.02,  # Minimum etki büyüklüğü
            'confidence_level': 0.95,  # Güven seviyesi (%95)
            'random_state': 42,  # Rastgele sayı sabiti
            'test_duration_days': 14,  # Test süresi (gün)
            'min_sample_size': 1000  # Minimum örnek boyutu
        }

    # Gerçekçi sentetik veri üretme
    def generate_realistic_data(self, n_samples: int = 10000) -> pd.DataFrame:
        # Rastgele sayı üretimini sabitleme
        np.random.seed(self.config['random_state'])
        print(f"{n_samples:,} kullanıcı için veri oluşturuluyor...")

        # Yaş: Normal dağılımla, 18-70 arası
        ages = np.random.normal(35, 12, n_samples)
        ages = np.clip(ages, 18, 70).astype(int)
        # Cinsiyet: %52 erkek, %48 kadın
        genders = np.random.choice(['Male', 'Female'], n_samples, p=[0.52, 0.48])
        # Bölge: İstanbul, Ankara vb. farklı olasılıklarla
        regions = np.random.choice(
            ['Istanbul', 'Ankara', 'Izmir', 'Bursa', 'Antalya', 'Other'],
            n_samples, p=[0.35, 0.15, 0.12, 0.08, 0.1, 0.2]
        )
        # Cihaz: Mobil, masaüstü, tablet
        devices = np.random.choice(['mobile', 'desktop', 'tablet'], n_samples, p=[0.65, 0.28, 0.07])
        # Grup: A veya B, eşit olasılıkla
        groups = np.random.choice(['A', 'B'], n_samples, p=[0.5, 0.5])

        # Önceki aktivite puanı: Gamma dağılımı, 0-10 arası
        previous_activity = np.random.gamma(2, 2, n_samples)
        previous_activity = np.clip(previous_activity, 0, 10)

        # Sayfada kalış süresi: Yaş, grup, cihaz ve bölge etkileriyle
        base_time = 90 + ages * 0.8 + previous_activity * 8
        group_effect = np.where(groups == 'B', 28, 0)  # Grup B için ek süre
        device_effect = np.where(devices == 'mobile', -15, np.where(devices == 'tablet', 8, 0))
        region_effect = np.where(regions == 'Istanbul', 5, 0)
        time_on_page = (base_time + group_effect + device_effect +
                       region_effect + np.random.normal(0, 25, n_samples))
        time_on_page = np.clip(time_on_page, 15, 500).astype(int)

        # Dönüşüm olasılığı: Lojistik modelle hesaplanır
        conversion_logit = (
            -2.5 + 0.4 * (groups == 'B') + 0.02 * (ages - 35) +
            0.3 * (genders == 'Female') + 0.1 * (devices == 'desktop') +
            0.05 * previous_activity + 0.002 * time_on_page
        )
        conversion_prob = 1 / (1 + np.exp(-conversion_logit))
        conversions = np.random.binomial(1, conversion_prob, n_samples)

        # Gelir: Sadece dönüşüm yapanlar için, log-normal dağılımla
        revenue = np.zeros(n_samples)
        converted_mask = conversions == 1
        n_converted = np.sum(converted_mask)
        if n_converted > 0:
            base_revenue = np.random.lognormal(4.2, 0.6, n_converted)
            group_revenue_multiplier = np.where(groups[converted_mask] == 'B', 1.12, 1.0)
            age_effect = 1 + (ages[converted_mask] - 35) * 0.008
            revenue[converted_mask] = (base_revenue * group_revenue_multiplier * age_effect)

        # Sayfa görüntülemeleri ve tıklamalar: Poisson dağılımı
        page_views = np.random.poisson(2.5 + 0.6 * (groups == 'B') + 0.1 * previous_activity, n_samples)
        clicks = np.random.poisson(1.8 + 0.4 * (groups == 'B') + 0.08 * previous_activity, n_samples)

        # Kayıt tarihleri: Son 14 gün içinde rastgele veya düzenli
        start_date = datetime.now() - timedelta(days=self.config['test_duration_days'])
        total_minutes = self.config['test_duration_days'] * 24 * 60
        if n_samples > total_minutes:
            random_minutes = np.random.randint(0, total_minutes, n_samples)
            dates = [start_date + timedelta(minutes=int(m)) for m in random_minutes]
        else:
            freq = total_minutes // n_samples
            dates = pd.date_range(start=start_date, periods=n_samples, freq=f'{freq}min')

        # Veriyi bir DataFrame'e toplama
        self.data = pd.DataFrame({
            'user_id': [f'user_{i:06d}' for i in range(1, n_samples + 1)],
            'group': groups,
            'age': ages,
            'gender': genders,
            'region': regions,
            'device': devices,
            'previous_activity_score': np.round(previous_activity, 2),
            'time_on_page_seconds': time_on_page,
            'page_views': page_views,
            'clicks': clicks,
            'conversion': conversions,
            'revenue_tl': np.round(revenue, 2),
            'signup_date': dates,
            'day_of_week': [d.day_name() for d in dates],
            'hour_of_day': [d.hour for d in dates]
        })

        # Veri özetini yazdırma
        print(f" Veri oluşturuldu: {len(self.data):,} kayıt")
        print(f" Grup dağılımı: A={np.sum(groups=='A'):,}, B={np.sum(groups=='B'):,}")
        print(f" Toplam dönüşüm: {np.sum(conversions):,} ({np.mean(conversions)*100:.1f}%)")
        print(f" Toplam gelir: {np.sum(revenue):,.0f} TL")
        return self.data

    # Verinin kalitesini kontrol etme
    def perform_data_quality_checks(self) -> Dict:
        # Veri yoksa hata fırlatır
        if self.data is None:
            raise ValueError(" Önce veri oluşturun veya yükleyin!")
        print("\n" + "="*50)
        print(" VERİ KALİTESİ KONTROLLÜ")
        print("="*50)

        quality_report = {}
        # Eksik verileri kontrol etme
        missing_data = self.data.isnull().sum()
        quality_report['missing_data'] = missing_data.to_dict()
        # Tekrar eden kayıtları kontrol etme
        duplicates = self.data.duplicated().sum()
        quality_report['duplicates'] = duplicates

        # Sayısal sütunlarda aykırı değerleri tespit etme
        numeric_cols = self.data.select_dtypes(include=[np.number]).columns
        outliers = {}
        for col in numeric_cols:
            Q1 = self.data[col].quantile(0.25)
            Q3 = self.data[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outlier_count = ((self.data[col] < lower_bound) | (self.data[col] > upper_bound)).sum()
            outliers[col] = outlier_count
        quality_report['outliers'] = outliers

        # Grup dengesini kontrol etme
        group_balance = self.data['group'].value_counts()
        balance_ratio = group_balance.min() / group_balance.max()
        quality_report['group_balance'] = {
            'counts': group_balance.to_dict(),
            'balance_ratio': balance_ratio
        }

        # Kalite özetini yazdırma
        print(f" Toplam kayıt sayısı: {len(self.data):,}")
        print(f" Eksik veri: {missing_data.sum()} adet")
        print(f" Duplicate kayıt: {duplicates} adet")
        print(f" Grup dengesi oranı: {balance_ratio:.3f}")
        if missing_data.sum() == 0:
            print(" Eksik veri yok!")
        if duplicates == 0:
            print(" Duplicate kayıt yok!")
        if balance_ratio > 0.95:
            print(" Grup dağılımı dengeli!")
        return quality_report

    # İstatistiksel testler
    def run_statistical_tests(self) -> Dict:
        # Veri yoksa hata fırlatma
        if self.data is None:
            raise ValueError(" Önce veri oluşturun!")
        print("\n" + "="*50)
        print(" İSTATİSTİKSEL TESTLER")
        print("="*50)

        test_results = {}
        group_a = self.data[self.data['group'] == 'A']
        group_b = self.data[self.data['group'] == 'B']

        # 1. Dönüşüm oranı analizi (Chi-square testi)
        print("\n 1 DÖNÜŞÜM ORANI ANALİZİ")
        print("-" * 30)
        contingency_table = pd.crosstab(self.data['group'], self.data['conversion'])
        chi2, p_chi2, dof, expected = chi2_contingency(contingency_table)
        conv_rate_a = group_a['conversion'].mean()
        conv_rate_b = group_b['conversion'].mean()
        relative_lift = ((conv_rate_b - conv_rate_a) / conv_rate_a) * 100
        test_results['conversion_analysis'] = {
            'chi2_statistic': float(chi2),
            'p_value': float(p_chi2),
            'degrees_of_freedom': int(dof),
            'conversion_rate_a': float(conv_rate_a),
            'conversion_rate_b': float(conv_rate_b),
            'absolute_lift': float(conv_rate_b - conv_rate_a),
            'relative_lift_percent': float(relative_lift),
            'is_significant': int(p_chi2 < self.config['alpha_level'])
        }
        print(f"   Grup A Dönüşüm Oranı: {conv_rate_a:.4f} ({conv_rate_a*100:.2f}%)")
        print(f"   Grup B Dönüşüm Oranı: {conv_rate_b:.4f} ({conv_rate_b*100:.2f}%)")
        print(f"   Relatif İyileştirme: {relative_lift:+.2f}%")
        print(f"   Chi-square: {chi2:.4f}, p-value: {p_chi2:.6f}")
        print(f"   Sonuç: {' İstatistiksel olarak anlamlı' if p_chi2 < 0.05 else ' Anlamlı değil'}")

        # 2. Sayfa kalış süresi analizi (T-test)
        print("\n 2 SAYFA KALIŞ SÜRESİ ANALİZİ")
        print("-" * 30)
        t_stat, p_ttest = ttest_ind(group_a['time_on_page_seconds'], group_b['time_on_page_seconds'], equal_var=False)
        levene_stat, levene_p = stats.levene(group_a['time_on_page_seconds'], group_b['time_on_page_seconds'])
        pooled_std = np.sqrt(((len(group_a)-1)*group_a['time_on_page_seconds'].var() +
                             (len(group_b)-1)*group_b['time_on_page_seconds'].var()) /
                            (len(group_a)+len(group_b)-2))
        cohens_d = (group_b['time_on_page_seconds'].mean() - group_a['time_on_page_seconds'].mean()) / pooled_std
        test_results['time_on_page_analysis'] = {
            't_statistic': float(t_stat),
            'p_value': float(p_ttest),
            'levene_p_value': float(levene_p),
            'cohens_d': float(cohens_d),
            'mean_a': float(group_a['time_on_page_seconds'].mean()),
            'mean_b': float(group_b['time_on_page_seconds'].mean()),
            'is_significant': int(p_ttest < self.config['alpha_level'])
        }
        print(f"   Grup A Ortalama: {group_a['time_on_page_seconds'].mean():.1f} saniye")
        print(f"   Grup B Ortalama: {group_b['time_on_page_seconds'].mean():.1f} saniye")
        print(f"   T-test: t={t_stat:.4f}, p-value={p_ttest:.6f}")
        print(f"   Cohen's d: {cohens_d:.3f}")
        print(f"   Sonuç: {' İstatistiksel olarak anlamlı' if p_ttest < 0.05 else ' Anlamlı değil'}")

        # 3. Gelir analizi (Mann-Whitney U testi)
        print("\n 3 GELİR ANALİZİ")
        print("-" * 30)
        mw_stat, p_mw = mannwhitneyu(group_a['revenue_tl'], group_b['revenue_tl'], alternative='two-sided')
        test_results['revenue_analysis'] = {
            'mw_statistic': float(mw_stat),
            'p_value': float(p_mw),
            'median_a': float(group_a['revenue_tl'].median()),
            'median_b': float(group_b['revenue_tl'].median()),
            'is_significant': int(p_mw < self.config['alpha_level'])
        }
        print(f"   Grup A Medyan Gelir: {group_a['revenue_tl'].median():.2f} TL")
        print(f"   Grup B Medyan Gelir: {group_b['revenue_tl'].median():.2f} TL")
        print(f"   Mann-Whitney U: stat={mw_stat:.4f}, p-value={p_mw:.6f}")
        print(f"   Sonuç: {' İstatistiksel olarak anlamlı' if p_mw < 0.05 else ' Anlamlı değil'}")

        # 4. Bayesçi A/B testi
        print("\n 4 BAYESÇİ A/B TESTİ")
        print("-" * 30)
        try:
            from scipy.stats import beta
            a_A, b_A = group_a['conversion'].sum() + 1, len(group_a) - group_a['conversion'].sum() + 1
            a_B, b_B = group_b['conversion'].sum() + 1, len(group_b) - group_b['conversion'].sum() + 1
            beta_A = beta.rvs(a_A, b_A, size=10000)
            beta_B = beta.rvs(a_B, b_B, size=10000)
            prob_B_better = np.mean(beta_B > beta_A)
            test_results['bayesian_analysis'] = {
                'probability_B_better': float(prob_B_better),
                'mean_conversion_A': float(a_A / (a_A + b_A)),
                'mean_conversion_B': float(a_B / (a_B + b_B))
            }
            print(f"   Grup B'nin daha iyi olma olasılığı: {prob_B_better:.3f}")
            print(f"   Ortalama Dönüşüm (A): {a_A / (a_A + b_A):.4f}")
            print(f"   Ortalama Dönüşüm (B): {a_B / (a_B + b_B):.4f}")
        except Exception as e:
            print(f"   Bayesçi test hatası: {str(e)}")
            test_results['bayesian_analysis'] = {'error': str(e)}

        # 5. Güç analizi
        print("\n 5 GÜÇ ANALİZİ")
        print("-" * 30)
        power_analysis = TTestIndPower()
        effect_size = abs(conv_rate_b - conv_rate_a) / np.sqrt((conv_rate_a * (1 - conv_rate_a) + conv_rate_b * (1 - conv_rate_b)) / 2)
        power = power_analysis.solve_power(effect_size=effect_size, nobs1=len(group_a), alpha=self.config['alpha_level'])
        test_results['power_analysis'] = {
            'effect_size': float(effect_size),
            'power': float(power),
            'sample_size_per_group': int(len(group_a))
        }
        print(f"   Etki Büyüklüğü: {effect_size:.3f}")
        print(f"   Test Gücü: {power:.3f} (>{self.config['power']} idealdir)")

        self.results['statistical_tests'] = test_results
        return test_results

    # Makine öğrenmesi modellerini eğitme
    def train_ml_models(self) -> Dict:
        # Veri yoksa hata fırlatma
        if self.data is None:
            raise ValueError(" Önce veri oluşturun!")
        print("\n" + "="*50)
        print(" MAKİNE ÖĞRENMESİ MODELLERİ")
        print("="*50)

        # Özellikleri seçme (interaction_score hariç, sonradan oluşturulacak)
        features = ['age', 'gender', 'region', 'device', 'previous_activity_score',
                    'time_on_page_seconds', 'page_views', 'clicks']
        X = self.data[features].copy()
        # Sayfada kalış süresini log dönüşümü uygulama
        X['time_on_page_seconds'] = np.log1p(X['time_on_page_seconds'])
        # Yeni özellik: interaction_score = sayfa görüntülemeleri * tıklamalar
        X['interaction_score'] = X['page_views'] * X['clicks']

        # Hedef değişken: dönüşüm
        target = 'conversion'
        y = self.data[target]

        # Kategorik değişkenleri sayısal değerlere dönüştürme
        le = LabelEncoder()
        for col in ['gender', 'region', 'device']:
            X[col] = le.fit_transform(X[col])

        # Veriyi eğitim ve test setlerine böler (%80 eğitim, %20 test)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=self.config['random_state'], stratify=y
        )

        # Özellikleri standartlaştırma
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Dengesiz sınıfları dengelemek için SMOTE uygulama
        smote = SMOTE(random_state=self.config['random_state'])
        X_train, y_train = smote.fit_resample(X_train, y_train)

        # Modelleri tanımlama
        models = {
            'Logistic Regression': LogisticRegression(random_state=self.config['random_state'], class_weight='balanced'),
            'Random Forest': RandomForestClassifier(
                random_state=self.config['random_state'], n_estimators=200, max_depth=5, min_samples_split=5
            ),
            'Gradient Boosting': GradientBoostingClassifier(
                random_state=self.config['random_state'], n_estimators=200, max_depth=5, learning_rate=0.05
            )
        }

        model_results = {}
        # Her modeli eğitme ve değerlendirme
        for name, model in models.items():
            print(f"\n {name} modeli eğitiliyor...")
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            y_prob = model.predict_proba(X_test)[:, 1]
            report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
            auc = roc_auc_score(y_test, y_prob)
            model_results[name] = {
                'classification_report': report,
                'roc_auc': float(auc),
                'confusion_matrix': confusion_matrix(y_test, y_pred).tolist()
            }
            print(f"   ROC AUC: {auc:.3f}")
            print(f"   Precision (1): {report['1']['precision']:.3f}")
            print(f"   Recall (1): {report['1']['recall']:.3f}")

        self.results['ml_models'] = model_results
        return model_results

    # Görselleştirmeler  ve HTML dosyalarını kaydedetme
    def create_visualizations(self, output_dir: str = 'results/') -> None:
        # Veri yoksa hata fırlatır
        if self.data is None:
            raise ValueError(" Önce veri oluşturun!")
        # Çıktı dizini yoksa oluşturma
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        print("\n" + "="*50)
        print(" GÖRSELLEŞTİRMELER")
        print("="*50)

        # Dönüşüm oranı çubuk grafiği
        conv_rates = self.data.groupby('group')['conversion'].mean()
        fig = px.bar(
            x=conv_rates.index,
            y=conv_rates.values,
            title='A/B Testi: Dönüşüm Oranı Karşılaştırması',
            labels={'x': 'Grup', 'y': 'Dönüşüm Oranı'},
            color=conv_rates.index,
            text=[f"{x:.2%}" for x in conv_rates.values]
        )
        fig.update_layout(showlegend=False)
        fig.update_yaxes(range=[0, max(conv_rates.values) + 0.05])
        fig.update_traces(textposition='auto')
        fig.write_html(os.path.join(output_dir, 'conversion_rates.html'))
        print(f" Dönüşüm oranı grafiği kaydedildi: {output_dir}conversion_rates.html")

        # Cihaz türüne göre segmentasyon grafiği
        segment_data = self.data.groupby(['group', 'device'])['conversion'].mean().unstack()
        fig = go.Figure(data=[
            go.Bar(name=device, x=segment_data.index, y=segment_data[device])
            for device in segment_data.columns
        ])
        fig.update_layout(
            title='Cihaz Türüne Göre Dönüşüm Oranları',
            xaxis_title='Grup',
            yaxis_title='Dönüşüm Oranı',
            barmode='group'
        )
        fig.write_html(os.path.join(output_dir, 'device_segmentation.html'))
        print(f" Segmentasyon grafiği kaydedildi: {output_dir}device_segmentation.html")

        # Gelir dağılımı kutu grafiği
        fig = px.box(
            self.data[self.data['revenue_tl'] > 0],
            x='group',
            y='revenue_tl',
            title='Grup Bazında Gelir Dağılımı (Dönüşen Kullanıcılar)',
            color='group'
        )
        fig.update_layout(yaxis_title='Gelir (TL)')
        fig.write_html(os.path.join(output_dir, 'revenue_distribution.html'))
        print(f" Gelir dağılımı grafiği kaydedildi: {output_dir}revenue_distribution.html")

    # İş metriklerini hesaplama
    def calculate_business_metrics(self) -> Dict:
        # Veri yoksa hata fırlatma
        if self.data is None:
            raise ValueError(" Önce veri oluşturun!")
        print("\n" + "="*50)
        print(" İŞ METRİKLERİ")
        print("="*50)

        metrics = {}
        group_a = self.data[self.data['group'] == 'A']
        group_b = self.data[self.data['group'] == 'B']
        # Dönüşüm oranı artışı
        conv_rate_a = group_a['conversion'].mean()
        conv_rate_b = group_b['conversion'].mean()
        lift = (conv_rate_b - conv_rate_a) / conv_rate_a * 100
        # Kullanıcı başına gelir
        rpu_a = group_a['revenue_tl'].mean()
        rpu_b = group_b['revenue_tl'].mean()
        rpu_lift = (rpu_b - rpu_a) / rpu_a * 100
        # Yatırım getirisi (ROI)
        test_cost = 5000
        expected_revenue_b = rpu_b * len(group_b)
        roi = ((expected_revenue_b - test_cost) / test_cost) * 100
        metrics = {
            'conversion_lift_percent': float(lift),
            'revenue_per_user_a': float(rpu_a),
            'revenue_per_user_b': float(rpu_b),
            'revenue_lift_percent': float(rpu_lift),
            'estimated_roi_percent': float(roi)
        }
        print(f"   Dönüşüm Artışı: {lift:.2f}%")
        print(f"   Kullanıcı Başına Gelir (A): {rpu_a:.2f} TL")
        print(f"   Kullanıcı Başına Gelir (B): {rpu_b:.2f} TL")
        print(f"   Gelir Artışı: {rpu_lift:.2f}%")
        print(f"   Tahmini ROI: {roi:.2f}%")
        self.results['business_metrics'] = metrics
        return metrics

    # Tüm analiz adımlarını çalıştırma
    def run_complete_analysis(self, output_dir: str = 'results/') -> Dict:
        print("\n" + "="*50)
        print(" TAM ANALİZ BAŞLIYOR")
        print("="*50)
        # Veri yoksa üretme
        if self.data is None:
            self.generate_realistic_data()
        # Kalite kontrolü
        self.perform_data_quality_checks()
        # İstatistiksel testleri çalıştırma
        self.run_statistical_tests()
        # Makine öğrenmesi modellerini eğitme
        self.train_ml_models()
        # Görselleştirmeler oluşturma
        self.create_visualizations(output_dir)
        # İş metriklerini hesaplar
        self.calculate_business_metrics()
        # Sonuçları JSON formatına çevirir ve kaydeder
        serializable_results = make_json_serializable(self.results)
        with open(os.path.join(output_dir, 'results.json'), 'w') as f:
            json.dump(serializable_results, f, indent=4)
        print(f" Sonuçlar kaydedildi: {output_dir}results.json")
        return self.results

    # Sonuçları JSON dosyasına kaydeder
    def save_results(self, output_dir: str = 'results/') -> None:
        # Çıktı dizini yoksa oluşturur
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        # Sonuçları JSON uyumlu hale getirir
        serializable_results = make_json_serializable(self.results)
        with open(os.path.join(output_dir, 'results.json'), 'w') as f:
            json.dump(serializable_results, f, indent=4)
        print(f" Sonuçlar kaydedildi: {output_dir}results.json")

# Unit testler: Kodun doğruluğunu kontrol eder
class TestDataGenerator(unittest.TestCase):
    # Veri üretimini test eder
    def test_data_generation(self):
        analyzer = ABTestAnalyzer()
        data = analyzer.generate_realistic_data(n_samples=100)
        self.assertEqual(len(data), 100)
        self.assertIn('conversion', data.columns)
        self.assertIn('signup_date', data.columns)
        self.assertIn('group', data.columns)
        self.assertTrue(all(data['group'].isin(['A', 'B'])))
        self.assertGreater(data['conversion'].mean(), 0)

    # Grup dengesini test eder
    def test_group_balance(self):
        analyzer = ABTestAnalyzer()
        data = analyzer.generate_realistic_data(n_samples=100)
        group_counts = data['group'].value_counts()
        balance_ratio = group_counts.min() / group_counts.max()
        self.assertGreater(balance_ratio, 0.8)  # Grup dengesi %80'den fazla

    # Dönüşüm oranı artışını test eder
    def test_conversion_lift(self):
        analyzer = ABTestAnalyzer()
        data = analyzer.generate_realistic_data(n_samples=100)
        conv_rate_a = data[data['group'] == 'A']['conversion'].mean()
        conv_rate_b = data[data['group'] == 'B']['conversion'].mean()
        self.assertGreater(conv_rate_b, conv_rate_a)  # Grup B daha yüksek dönüşüm

    # Veri kalitesini test eder
    def test_data_quality(self):
        analyzer = ABTestAnalyzer()
        data = analyzer.generate_realistic_data(n_samples=100)
        self.assertEqual(data.isnull().sum().sum(), 0)  # Eksik veri yok
        self.assertEqual(data.duplicated().sum(), 0)   # Tekrar eden kayıt yok

    # Gelir dağılımını test eder
    def test_revenue_distribution(self):
        analyzer = ABTestAnalyzer()
        data = analyzer.generate_realistic_data(n_samples=100)
        revenue_converted = data[data['conversion'] == 1]['revenue_tl']
        self.assertGreater(revenue_converted.mean(), 0)  # Dönüşenlerin geliri pozitif

    # Makine öğrenmesi performansını test eder
    def test_ml_performance(self):
        analyzer = ABTestAnalyzer()
        data = analyzer.generate_realistic_data(n_samples=1000)
        results = analyzer.train_ml_models()
        for model_name, metrics in results.items():
            self.assertGreater(metrics['roc_auc'], 0.5)  # ROC AUC > 0.5

# Testleri ve tam analizi çalıştırır
try:
    # Unit testleri çalıştırır
    print("\n Unit Testler Çalıştırılıyor...")
    unittest.main(argv=[''], exit=False)

    # Tam analizi başlatır
    analyzer = ABTestAnalyzer()
    results = analyzer.run_complete_analysis()

    # Sonuçları JSON dosyasından okur ve yazdırır
    print("\n Sonuçlar:")
    with open('results/results.json', 'r') as f:
        print(json.load(f))

    # Görselleştirme dosyalarını indirir (Google Colab için)
    from google.colab import files
    for file in ['conversion_rates.html', 'device_segmentation.html', 'revenue_distribution.html']:
        file_path = os.path.join('results', file)
        if os.path.exists(file_path):
            files.download(file_path)
            print(f" {file} indirildi")
        else:
            print(f"Uyarı: {file} bulunamadı!")
except Exception as e:
    # Hata olursa mesajı ve ayrıntıları yazdırır
    print(f"Hata oluştu: {str(e)}")
    import traceback
    traceback.print_exc()

  Cloning https://github.com/Matt52/bayesian-testing.git to /tmp/pip-req-build-2c_wxnbl
  Running command git clone --filter=blob:none --quiet https://github.com/Matt52/bayesian-testing.git /tmp/pip-req-build-2c_wxnbl
  Resolved https://github.com/Matt52/bayesian-testing.git to commit cea9afa5d7e3321d159d7b387ff57803467a18d5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for bayesian-testing: filename=bayesian_testing-0.9.1-py3-none-any.whl size=33344 sha256=602e99a3783b7abebea44cbeff0d8cd22388bdddfbe0cf675b3a83df5bd9d3ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-0zp8x9qw/wheels/87/c4/b3/03ee000efc07699d97e6d89880515babe51416b41b59eec945
Successfully built bayesian-testing


....


 Unit Testler Çalıştırılıyor...
 A/B Test Analyzer başlatıldı!
 Konfigürasyon: 7 ayar yüklendi
100 kullanıcı için veri oluşturuluyor...
 Veri oluşturuldu: 100 kayıt
 Grup dağılımı: A=45, B=55
 Toplam dönüşüm: 14 (14.0%)
 Toplam gelir: 1,074 TL
 A/B Test Analyzer başlatıldı!
 Konfigürasyon: 7 ayar yüklendi
100 kullanıcı için veri oluşturuluyor...
 Veri oluşturuldu: 100 kayıt
 Grup dağılımı: A=45, B=55
 Toplam dönüşüm: 14 (14.0%)
 Toplam gelir: 1,074 TL
 A/B Test Analyzer başlatıldı!
 Konfigürasyon: 7 ayar yüklendi
100 kullanıcı için veri oluşturuluyor...
 Veri oluşturuldu: 100 kayıt
 Grup dağılımı: A=45, B=55
 Toplam dönüşüm: 14 (14.0%)
 Toplam gelir: 1,074 TL
 A/B Test Analyzer başlatıldı!
 Konfigürasyon: 7 ayar yüklendi
100 kullanıcı için veri oluşturuluyor...
 Veri oluşturuldu: 100 kayıt
 Grup dağılımı: A=45, B=55
 Toplam dönüşüm: 14 (14.0%)
 Toplam gelir: 1,074 TL
 A/B Test Analyzer başlatıldı!
 Konfigürasyon: 7 ayar yüklendi
1,000 kullanıcı için veri oluşturuluyor...
 Veri oluştur

F.
FAIL: test_ml_performance (__main__.TestDataGenerator.test_ml_performance)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-1-143310588>", line 592, in test_ml_performance
    self.assertGreater(metrics['roc_auc'], 0.5)  # ROC AUC > 0.5
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: 0.46130952380952384 not greater than 0.5

----------------------------------------------------------------------
Ran 6 tests in 2.446s

FAILED (failures=1)


   ROC AUC: 0.471
   Precision (1): 0.000
   Recall (1): 0.000
 A/B Test Analyzer başlatıldı!
 Konfigürasyon: 7 ayar yüklendi
100 kullanıcı için veri oluşturuluyor...
 Veri oluşturuldu: 100 kayıt
 Grup dağılımı: A=45, B=55
 Toplam dönüşüm: 14 (14.0%)
 Toplam gelir: 1,074 TL
 A/B Test Analyzer başlatıldı!
 Konfigürasyon: 7 ayar yüklendi

 TAM ANALİZ BAŞLIYOR
10,000 kullanıcı için veri oluşturuluyor...
 Veri oluşturuldu: 10,000 kayıt
 Grup dağılımı: A=5,022, B=4,978
 Toplam dönüşüm: 1,661 (16.6%)
 Toplam gelir: 144,145 TL

 VERİ KALİTESİ KONTROLLÜ
 Toplam kayıt sayısı: 10,000
 Eksik veri: 0 adet
 Duplicate kayıt: 0 adet
 Grup dengesi oranı: 0.991
 Eksik veri yok!
 Duplicate kayıt yok!
 Grup dağılımı dengeli!

 İSTATİSTİKSEL TESTLER

 1 DÖNÜŞÜM ORANI ANALİZİ
------------------------------
   Grup A Dönüşüm Oranı: 0.1438 (14.38%)
   Grup B Dönüşüm Oranı: 0.1886 (18.86%)
   Relatif İyileştirme: +31.20%
   Chi-square: 36.0027, p-value: 0.000000
   Sonuç:  İstatistiksel olarak anlamlı

 2 SAY

...

<IPython.core.display.Javascript object>

 device_segmentation.html indirildi


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 revenue_distribution.html indirildi
